In [76]:
from keras import *
from keras.utils import *
from keras.datasets import *
from keras.layers import *
from keras.engine import *
from tensorflow.examples.tutorials.mnist import input_data
from keras.models import *
from numpy import *
from keras.activations import *


class DCGAN:
    def __init__(self,img_row = 28,img_col = 28,channal = 1):
        self.img_col = img_col
        self.img_row = img_row
        self.channal = channal
        
        self.D = None
        self.G = None
        self.AM = None
        self.DM = None
        
    def discriminator(self):
        if(self.D):
            return self.D
        self.D = Sequential();
        depth = 64;
        dropout = 0.5
        
        input_shape = (self.img_row,self.img_col,self.channal)
        self.D.add(Conv2D(depth*1,5,strides=1,input_shape=input_shape,activation='relu',padding='same'))
        self.D.add(BatchNormalization(momentum=0.9))
        self.D.add(Dropout(dropout))
        
        self.D.add(Conv2D(depth*2,5,strides=1,activation='relu',padding='same'))
        self.D.add(BatchNormalization(momentum=0.9))
        
        self.D.add(Flatten())
        self.D.add(Dense(1,activation='sigmoid'))
        
        self.D.summary()
        return self.D
    
    def Generator(self):
        if(self.G):
            return self.G
        self.G = Sequential()
        dropout=0.5
        depth = 64*4
        dim = 7
        
        self.G.add(Dense(dim*dim*depth,activation='relu',input_dim=100))
        self.G.add(Reshape((dim,dim,depth)))

        
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/2), 5, padding='same',activation='relu'))
        
        self.G.add(UpSampling2D())
        self.G.add(Conv2DTranspose(int(depth/4),5,padding='same',activation='relu'))
        
        self.G.add(Conv2DTranspose(1,5,padding='same',activation='sigmoid'))
        
        self.G.summary()
        
        return self.G
    
    def discrimation_model(self):
        if(self.DM):
            return self.DM
        self.DM=Sequential()
        self.DM.add(self.discriminator())
        self.DM.compile(optimizer='RMSprop',loss='mse',metrics=['accuracy'])
        
        return self.DM
    
    def generator_model(self):
        if(self.AM):
            return self.AM
        self.AM = Sequential()
        
        self.AM.add(self.Generator())
        self.AM.add(self.discriminator())
    
        self.AM.compile(optimizer='RMSprop',loss='mse',metrics=['accuracy'])
        
        return self.AM

Extracting mnist\train-images-idx3-ubyte.gz


Extracting mnist\train-labels-idx1-ubyte.gz
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz


In [77]:
x_train = input_data.read_data_sets("mnist",\
        	one_hot=True).train.images
x_train = x_train.reshape(-1, 28,\
        	28, 1).astype(np.float32)

step = 100
batch_size = 256

Extracting mnist\train-images-idx3-ubyte.gz


Extracting mnist\train-labels-idx1-ubyte.gz
Extracting mnist\t10k-images-idx3-ubyte.gz
Extracting mnist\t10k-labels-idx1-ubyte.gz


In [79]:
DC = DCGAN()
dm = DC.discrimation_model()
am = DC.generator_model()
gen =DC.Generator()

for i in range(step):
    images_train = x_train[np.random.randint(0,
                x_train.shape[0], size=batch_size), :, :, :]
    noise = random.uniform(-1.0, 1.0, size=[batch_size, 100])

    y=ones([batch_size,1])
    am.fit(noise,y,epochs=1,batch_size=256)
    
    images_fake = gen.predict(noise)
    x = concatenate((images_train, images_fake))
    y = ones([2*batch_size, 1])
    y[batch_size:, :] = 0
    #print(images_fake.shape)
    dm.fit(x,y,epochs=1,batch_size=512)
    print(dm.predict(x))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_53 (Conv2D)           (None, 28, 28, 64)        1664      
_________________________________________________________________
batch_normalization_3 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
dropout_27 (Dropout)         (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 28, 28, 128)       204928    
_________________________________________________________________
batch_normalization_4 (Batch (None, 28, 28, 128)       512       
_________________________________________________________________
flatten_15 (Flatten)         (None, 100352)            0         
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 100353    
Total para

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 12544)             1266944   
_________________________________________________________________
reshape_14 (Reshape)         (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_24 (UpSampling (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_transpose_53 (Conv2DT (None, 14, 14, 128)       819328    
_________________________________________________________________
up_sampling2d_25 (UpSampling (None, 28, 28, 128)       0         
_________________________________________________________________
conv2d_transpose_54 (Conv2DT (None, 28, 28, 64)        204864    
_________________________________________________________________
conv2d_transpose_55 (Conv2DT (None, 28, 28, 32)        51232     
__________

Epoch 1/1


256/256 [==============================] - 27s - loss: 0.7167 - acc: 0.0430

Epoch 1/1


...

Epoch 1/1


512/512 [==============================] - 19s - loss: 0.0000e+00 - acc: 1.0000

[[  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]
 [  1.00000000e+00]


256/256 [==============================] - 22s - loss: 0.0000e+00 - acc: 1.0000

KeyboardInterrupt: 